# Challenge SD-TSIA210

## TANG Joël & TORRES Claudia

In [2]:
))

import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import plit

Using tensorflow version 2.2.0-rc2


In [3]:
#%%
# Load training data

nrows_train = 1068504
nrows_test = 0

from google.colab import drive
drive.mount('/content/gdrive')
root_dir = "/content/gdrive/My Drive/"

xtrain = np.loadtxt(root_dir + 'xtrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_test)
ytrain = np.loadtxt(root_dir + 'ytrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_test)
ytrain = np.array(ytrain).reshape(nrows_train + nrows_test)
# Check the number of observations and properties
#print(xtrain[:3,])
#print(ytrain[:10])
#print(xtrain.shape)
#print(ytrain.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In the cell below, we perform data augmentation techniques.

Our algorithm (decision trees) doesn't require scaled data, and allow their range to be heterogeneous. However, we can still mirror the data: since each sample is composed of two faces, and a certain range of features is separable with respect to one of the two faces, we can simply exchange them. This increases the size of the training set by a 2 factor.

In [4]:
6:]
xtrain = xtrain.astype('float32')
xtest = np.loadtxt(root_dir + 'xtest_challenge.csv', delimiter=',', skiprows = 1).astype('floatain)

x_train_permuter = np.copy(xtrain)
x_train_permuter[:, :13] = xtrain[:, 13:26]
x_train_permuter[:, 13:26] = xtrain[:, :13]

new_x_train = np.concatenate((xtrain, x_train_permuter), axis=0)
new_y_train = np.concatenate((ytrain, ytrain), axisain)
xtrain_preprocessed = xtest)
xtest_preprocessextest
KAZE
print(xtrain_preprocessed.shape)
print(ytrain.shape)

(1068504, 37)
(1068504,)


In the code cell below we apply cross-validation on the XGBClassifier model. We boost the learning of many decision trees which are considered as weak leaners.
The most important parameters are :
- learning_rate: similar to other gradient descent methods, the classifier perform more or less a gradient correction. A small learning rate will lead to very slow convergence, a big learning rate can make us miss the sweet spot.
- n_estimators: to be considered as a trade-off with the learning rate. It is the number of rounds performed by the classifier during learning. A too great number will lead to over-fitting.
- max_depth: the maximum depth related to the decision trees. The less it is, the more the decision trees are basic (hence having higher bias, which is not a problem for boosting).

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

import tensorflow as tf

with tf.device('/device:GPU:0'):
    boost = XGBClassifier(base_score=0.45, colsample_bylevel=1, colsample_bytree=1,max_depth=6,
                          gamma=0, learning_rate=0.25, max_delta_step=0,
                          min_child_weight=1, missing=None, n_estimators=675, nthread=-1,
                          objective='binary:logistic', sampling_method='gradient_based',
                          reg_alpha=0, reg_lambda=1, tree_method='gpu_hist',
                          scale_pos_weight=1.002, silent=True, subsample=1)
  
    p_grid_boost = {'gamma': [0.5, 1, 1.5, 2],
                    'reg_lambda': [1.5, 2, 2.5, 3],
                    'colsample_bytree': [0.7, 0.85, 1.0],
                    'base_score': [0.45, 0.5],
                    'n_estimators': [650,655,660,665,670,675,680,685,690,695,700],
                    'learning_rate': [0.15, 0.2, 0.25,0.35]
                    }   

    grid_boost = RandomizedSearchCV(estimator=boost, param_distributions=p_grid_boost, scoring="accuracy", cv=5, verbose=3)
  
    grid_boost.fit(xtrain_preprocessed, ytrain)

    print("Best Score: {}".format(grid_boost.best_score_))
    print("Best params: {}".format(grid_boost.best_params_))

Best Score: 0.9990847015822186
Best params: {'max_depth': 6, 'n_estimators': 700}


In [7]:
boost = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                      gamma=0, learning_rate=0.25, max_delta_step=0, max_depth=6,
                      min_child_weight=1, missing=None, n_estimators=700, nthread=-1,
                      objective='binary:logistic', sampling_method='gradient_based',
                      reg_alpha=0, reg_lambda=1, tree_method='gpu_hist',
                      scale_pos_weight=1, seed=0, silent=True, subsample=1)
boost.fit(new_x_train, new_y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.25, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, sampling_method='gradient_based',
              scale_pos_weight=1, seed=0, silent=True, subsample=1,
              tree_method='gpu_hist', verbosity=1)

In [0]:
output = boost.predict(xtest_preprocessed)

In [0]:
np.savetxt(root_dir + 'ytest_challenge_student_cv.csv', output, fmt = '%1.0d', delimiter=',')